# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
import tensorflow as tf

from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


/Users/davidlaredorazo/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0.4

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(0.1), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness

    
    #Shared parameters for the models
    optimizer = Adam(lr=0,beta_1=0.5)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models
    model = RULmodel_SN_5(shape)
    model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

    return model




In [6]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)


# Load Data

In [7]:
tModel.data_handler.sequence_length = 24
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 129

In [8]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.2)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 24, stride of 1 and maxRUL of 129. Cros-Validation ratio 0.2
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(14738, 336)
(14738, 1)
Cross-Validation data (X, y)
(20, 336)
(20, 1)
Testing data (X, y)
(100, 336)
(100, 1)
Printing first 5 elements

Training data (X, y)
[[-0.44295302 -0.32685422 -0.37676609 ... -0.63636364  0.03875969
   0.39314799]
 [-0.02684564 -0.18158568 -0.35753532 ... -0.63636364  0.17829457
   0.42151081]
 [-0.42281879 -0.50332481 -0.26020408 ... -0.63636364  0.34883721
   0.07413648]
 [-0.13422819 -0.53554987 -0.46978022 ... -0.45454545  0.28682171
   0.35551811]
 [-0.48993289 -0.46598465 -0.53689168 ... -0.45454545  0.19379845
   0.44622297]]
[[129.]
 [129.]
 [129.]
 [129.]
 [129.]]
Cross-Validation data (X, y)
[[ 0.00671141 -0.11202046 -0.20800628 ... -0.63636364 -0.1627907
   0.25414209]
 [-0.19463087 -0.2629156  -0.31475667 ... -0.45454545  0.25581395
   0.43583263]
 [ 0.20805

# Train the model and test some Tunable Model functionalities

In [9]:
nFeatures = len(selected_features)

lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = nFeatures*tModel.data_handler.sequence_length


model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')


tModel.epochs = 200

sess = tf.Session()

tModel.train_model(learningRate_scheduler=lrate, verbose=1)

training with cv
Train on 14738 samples, validate on 20 samples
Epoch 1/200
14738/14738 [==============================] - 0s 28us/step - loss: 8321.9466 - mean_squared_error: 8298.5884 - val_loss: 12782.1387 - val_mean_squared_error: 12757.7080
Epoch 2/200
14738/14738 [==============================] - 0s 10us/step - loss: 6188.5992 - mean_squared_error: 6161.5292 - val_loss: 8316.6016 - val_mean_squared_error: 8285.7373
Epoch 3/200
14738/14738 [==============================] - 0s 10us/step - loss: 2927.5910 - mean_squared_error: 2892.2183 - val_loss: 3852.5461 - val_mean_squared_error: 3812.7407
Epoch 4/200
14738/14738 [==============================] - 0s 8us/step - loss: 1357.0860 - mean_squared_error: 1315.0553 - val_loss: 2112.1934 - val_mean_squared_error: 2068.5957
Epoch 5/200
14738/14738 [==============================] - 0s 8us/step - loss: 858.9078 - mean_squared_error: 814.5180 - val_loss: 1490.2571 - val_mean_squared_error: 1445.3029
Epoch 6/200
14738/14738 [=============

14738/14738 [==============================] - 0s 7us/step - loss: 295.1219 - mean_squared_error: 242.0289 - val_loss: 1187.1406 - val_mean_squared_error: 1134.3561
Epoch 47/200
14738/14738 [==============================] - 0s 7us/step - loss: 293.6972 - mean_squared_error: 241.1168 - val_loss: 1187.9762 - val_mean_squared_error: 1135.6321
Epoch 48/200
14738/14738 [==============================] - 0s 7us/step - loss: 292.3437 - mean_squared_error: 240.2298 - val_loss: 1135.8352 - val_mean_squared_error: 1083.9812
Epoch 49/200
14738/14738 [==============================] - 0s 7us/step - loss: 291.3738 - mean_squared_error: 239.7755 - val_loss: 1140.5223 - val_mean_squared_error: 1089.1677
Epoch 50/200
14738/14738 [==============================] - 0s 7us/step - loss: 290.1068 - mean_squared_error: 239.0175 - val_loss: 1147.3268 - val_mean_squared_error: 1096.5081
Epoch 51/200
14738/14738 [==============================] - 0s 6us/step - loss: 289.1501 - mean_squared_error: 238.5380 - v

14738/14738 [==============================] - 0s 6us/step - loss: 263.7958 - mean_squared_error: 224.9637 - val_loss: 1189.1454 - val_mean_squared_error: 1150.3683
Epoch 92/200
14738/14738 [==============================] - 0s 6us/step - loss: 263.3469 - mean_squared_error: 224.6519 - val_loss: 1186.2175 - val_mean_squared_error: 1147.6581
Epoch 93/200
14738/14738 [==============================] - 0s 7us/step - loss: 263.6447 - mean_squared_error: 225.1080 - val_loss: 1240.1346 - val_mean_squared_error: 1201.6952
Epoch 94/200
14738/14738 [==============================] - 0s 7us/step - loss: 262.8495 - mean_squared_error: 224.4819 - val_loss: 1181.5387 - val_mean_squared_error: 1143.2272
Epoch 95/200
14738/14738 [==============================] - 0s 6us/step - loss: 261.7715 - mean_squared_error: 223.5234 - val_loss: 1140.5387 - val_mean_squared_error: 1102.3190
Epoch 96/200
14738/14738 [==============================] - 0s 7us/step - loss: 262.0923 - mean_squared_error: 223.9614 - v

Epoch 137/200
14738/14738 [==============================] - 0s 6us/step - loss: 256.8370 - mean_squared_error: 219.7831 - val_loss: 1193.8151 - val_mean_squared_error: 1156.7677
Epoch 138/200
14738/14738 [==============================] - 0s 7us/step - loss: 256.7491 - mean_squared_error: 219.6989 - val_loss: 1188.0486 - val_mean_squared_error: 1151.0116
Epoch 139/200
14738/14738 [==============================] - 0s 7us/step - loss: 256.6654 - mean_squared_error: 219.6300 - val_loss: 1209.4231 - val_mean_squared_error: 1172.4192
Epoch 140/200
14738/14738 [==============================] - 0s 7us/step - loss: 256.6633 - mean_squared_error: 219.6484 - val_loss: 1191.7128 - val_mean_squared_error: 1154.7089
Epoch 141/200
14738/14738 [==============================] - 0s 7us/step - loss: 256.6544 - mean_squared_error: 219.6594 - val_loss: 1183.2083 - val_mean_squared_error: 1146.2043
Epoch 142/200
14738/14738 [==============================] - 0s 7us/step - loss: 256.5843 - mean_squared_

Epoch 183/200
14738/14738 [==============================] - 0s 7us/step - loss: 254.8115 - mean_squared_error: 218.4344 - val_loss: 1194.4237 - val_mean_squared_error: 1158.0602
Epoch 184/200
14738/14738 [==============================] - 0s 7us/step - loss: 254.8362 - mean_squared_error: 218.4721 - val_loss: 1169.6429 - val_mean_squared_error: 1133.2858
Epoch 185/200
14738/14738 [==============================] - 0s 6us/step - loss: 254.6668 - mean_squared_error: 218.3331 - val_loss: 1178.2551 - val_mean_squared_error: 1141.9113
Epoch 186/200
14738/14738 [==============================] - 0s 7us/step - loss: 254.5876 - mean_squared_error: 218.2620 - val_loss: 1210.5962 - val_mean_squared_error: 1174.2864
Epoch 187/200
14738/14738 [==============================] - 0s 7us/step - loss: 254.5601 - mean_squared_error: 218.2512 - val_loss: 1182.8966 - val_mean_squared_error: 1146.5817
Epoch 188/200
14738/14738 [==============================] - 0s 7us/step - loss: 254.5315 - mean_squared_

In [10]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=True)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

20/20 [==============================] - 0s 40us/step
scores
Engine 1, Predicted RUL 124.41279602050781, Rounded RUL 124.0, Real RUL 112.0
Engine 2, Predicted RUL 146.202392578125, Rounded RUL 146.0, Real RUL 98.0
Engine 3, Predicted RUL 133.16619873046875, Rounded RUL 133.0, Real RUL 69.0
Engine 4, Predicted RUL 31.11128044128418, Rounded RUL 31.0, Real RUL 82.0
Engine 5, Predicted RUL 106.24877166748047, Rounded RUL 106.0, Real RUL 91.0
Engine 6, Predicted RUL 118.80476379394531, Rounded RUL 118.0, Real RUL 93.0
Engine 7, Predicted RUL 102.56932067871094, Rounded RUL 102.0, Real RUL 91.0
Engine 8, Predicted RUL 108.11844635009766, Rounded RUL 108.0, Real RUL 95.0
Engine 9, Predicted RUL 26.169109344482422, Rounded RUL 26.0, Real RUL 111.0
Engine 10, Predicted RUL 80.682861328125, Rounded RUL 80.0, Real RUL 96.0
Engine 11, Predicted RUL 93.05181121826172, Rounded RUL 93.0, Real RUL 97.0
Engine 12, Predicted RUL 122.34523010253906, Rounded RUL 122.0, Real RUL 124.0
Engine 13, Predicted

In [11]:
tModel.evaluate_model(['rhs', 'rmse'], round=2, cross_validation=False)
print("scores")

cScores = tModel.scores
#rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, np.ravel(tModel.y_predicted), tModel.y_predicted_rounded, np.ravel(tModel.y_test)):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
#print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 92us/step
scores
Engine 1, Predicted RUL 111.69953918457031, Rounded RUL 111.0, Real RUL 112.0
Engine 2, Predicted RUL 128.86669921875, Rounded RUL 128.0, Real RUL 98.0
Engine 3, Predicted RUL 48.9747314453125, Rounded RUL 48.0, Real RUL 69.0
Engine 4, Predicted RUL 79.33831024169922, Rounded RUL 79.0, Real RUL 82.0
Engine 5, Predicted RUL 89.0667724609375, Rounded RUL 89.0, Real RUL 91.0
Engine 6, Predicted RUL 110.11637115478516, Rounded RUL 110.0, Real RUL 93.0
Engine 7, Predicted RUL 107.16773223876953, Rounded RUL 107.0, Real RUL 91.0
Engine 8, Predicted RUL 93.62696075439453, Rounded RUL 93.0, Real RUL 95.0
Engine 9, Predicted RUL 117.41218566894531, Rounded RUL 117.0, Real RUL 111.0
Engine 10, Predicted RUL 96.13442993164062, Rounded RUL 96.0, Real RUL 96.0
Engine 11, Predicted RUL 85.25056457519531, Rounded RUL 85.0, Real RUL 97.0
Engine 12, Predicted RUL 97.95358276367188, Rounded RUL 97.0, Real RUL 124.0
Engine 13, Predicted RUL 9